## GraphRAG

In [1]:
# !pip install langchain
# !pip install langchain_openai
# !pip install neo4j

In [6]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI

import os
from dotenv import load_dotenv
load_dotenv()

True

### 설정

In [7]:
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
USERNAME = os.environ.get("USERNAME")
PASSWORD = os.environ.get("PASSWORD")
URL = os.environ.get("URL")

In [ ]:
# llm test

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0,
                 max_tokens=2048,
                 model_name='gpt-3.5-turbo',
                )

question = 'ChatGPT가 뭐야?'

print(llm.predict(question))

ChatGPT는 OpenAI가 개발한 자연어 처리 기술을 기반으로 한 대화형 인공지능 챗봇입니다. ChatGPT는 사용자와 자연스럽게 대화하며 질문에 답변하거나 대화를 이어나갈 수 있습니다. 이를 통해 다양한 주제에 대한 정보를 제공하거나 사용자의 질문에 도움을 줄 수 있습니다.


### Neo4jGraph test

In [9]:
from langchain.chains import GraphCypherQAChain
from langchain_community.graphs import Neo4jGraph
from langchain_openai import ChatOpenAI

In [10]:
# graph = Neo4jGraph(url="bolt://", username="neo4j", password="") # url: bolt:// ~ , sandbox에서 확인한 log in info
graph = Neo4jGraph(url=URL, username=USERNAME, password=PASSWORD)

In [40]:
# LangChain이 내부적으로 Neo4j 그래프 스키마와 LLM(대규모 언어 모델)을 활용하여 해당 질문을 Cypher 쿼리로 변환하고, 그 결과를 다시 자연어 답변으로 생성
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(model_name="gpt-5-mini", temperature=0), graph=graph, verbose=True, allow_dangerous_requests=True
)

In [41]:
chain.run("Bella Roma 레스토랑의 주중 영업시간을 알려줘.")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (r:Restaurant {name: "Bella Roma"}) RETURN r.hoursWeekday AS hoursWeekday;
Full Context:
[{'hoursWeekday': '11:00-22:00'}]

> Finished chain.


'Bella Roma 레스토랑의 주중 영업시간은 11:00-22:00입니다.'

In [34]:
chain.run("까르보나라 파스타의 가격은 얼마야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:MenuItem {name: "까르보나라 파스타"}) RETURN m.price
Full Context:
[{'m.price': 18000}]

> Finished chain.


'까르보나라 파스타의 가격은 18,000원입니다.'

In [35]:
chain.run("바리스타로 일하는 직원은 누구야?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (e:Employee {role: "바리스타"}) RETURN e.name
Full Context:
[{'e.name': '안다혜'}]

> Finished chain.


'바리스타로 일하는 직원은 안다혜입니다.'

In [42]:
chain.run("Bella Roma에서 일하는 사람들을 확인해줘")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (e:Employee)-[:WORKS_AT_RESTAURANT]->(r:Restaurant {name: "Bella Roma"})
RETURN e.name AS name, e.role AS role, e.experienceYears AS experienceYears, e.workingDays AS workingDays;
Full Context:
[{'name': '김철수', 'role': '주방장(Head Chef)', 'experienceYears': 15, 'workingDays': '월~금'}, {'name': '이영희', 'role': '매니저', 'experienceYears': 10, 'workingDays': '월~토'}, {'name': '박민수', 'role': '서버', 'experienceYears': 2, 'workingDays': '화~일'}, {'name': '안다혜', 'role': '바리스타', 'experienceYears': 3, 'workingDays': '월~금'}]

> Finished chain.


'김철수, 이영희, 박민수, 안다혜가 Bella Roma에서 일합니다.'

In [43]:
chain.run("Bella Roma에서 일하는 사람들을 확인해주고, 그 중에서 바리스타인 사람을 찾아줘")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (r:Restaurant {name: "Bella Roma"})<-[:WORKS_AT_RESTAURANT]-(e:Employee)
WITH collect(e) AS employees
RETURN employees, [x IN employees WHERE x.role = "바리스타"] AS baristas;
Full Context:
[{'employees': [{'role': '주방장(Head Chef)', 'workingDays': '월~금', 'name': '김철수', 'experienceYears': 15}, {'role': '매니저', 'workingDays': '월~토', 'name': '이영희', 'experienceYears': 10}, {'role': '서버', 'workingDays': '화~일', 'name': '박민수', 'experienceYears': 2}, {'role': '바리스타', 'workingDays': '월~금', 'name': '안다혜', 'experienceYears': 3}], 'baristas': [{'role': '바리스타', 'workingDays': '월~금', 'name': '안다혜', 'experienceYears': 3}]}]

> Finished chain.


'Bella Roma에서 일하는 사람들: 김철수(주방장, 월~금, 경력 15년), 이영희(매니저, 월~토, 경력 10년), 박민수(서버, 화~일, 경력 2년), 안다혜(바리스타, 월~금, 경력 3년).  \n그중 바리스타인 사람은 안다혜입니다.'

### gpt-4o-mini, gpt-5-mini 성능 차이 (thinking)

In [ ]:
# gpt-4o-mini, gpt-5-mini 성능 차이 (thinking)
chain.run("Bella Roma에서 일하는 사람들을 확인해주고, 그 중에서 주방장을 찾아줘")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (r:Restaurant {name: "Bella Roma"})<-[:WORKS_AT_RESTAURANT]-(e:Employee)
WITH collect(e) AS employees
RETURN employees, [x IN employees WHERE x.role = "주방장"] AS headChefs
Full Context:
[{'employees': [{'role': '주방장(Head Chef)', 'workingDays': '월~금', 'name': '김철수', 'experienceYears': 15}, {'role': '매니저', 'workingDays': '월~토', 'name': '이영희', 'experienceYears': 10}, {'role': '서버', 'workingDays': '화~일', 'name': '박민수', 'experienceYears': 2}, {'role': '바리스타', 'workingDays': '월~금', 'name': '안다혜', 'experienceYears': 3}], 'headChefs': []}]

> Finished chain.


'Bella Roma에서 일하는 사람들: 김철수(주방장, 월~금, 경력 15년), 이영희(매니저, 월~토, 경력 10년), 박민수(서버, 화~일, 경력 2년), 안다혜(바리스타, 월~금, 경력 3년).  \n주방장은 김철수입니다.'

In [45]:
chain.run("티라미수를 구매한 이력이 있는 사용자들이 남긴 리뷰를 모두 알려줘")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Purchase)-[:FOR_MENU_ITEM]->(m:MenuItem {name: '티라미수'}),
      (p)-[:BY_CUSTOMER]->(c:Customer),
      (r:CustomerReview)-[:WRITTEN_BY]->(c)
RETURN DISTINCT r
Full Context:
[{'r': {'reviewDate': neo4j.time.Date(2024, 10, 3), 'author': '사용자1', 'rating': 4, 'caption': 'r사용자1|10-03|1', 'text': '직원분들이 친절해서 기분 좋게 식사했습니다.', 'seq': 1}}, {'r': {'reviewDate': neo4j.time.Date(2024, 10, 17), 'author': '사용자1', 'rating': 5, 'caption': 'r사용자1|10-17|1', 'text': '직원분들이 친절해서 기분 좋게 식사했습니다.', 'seq': 1}}, {'r': {'reviewDate': neo4j.time.Date(2024, 11, 10), 'author': '사용자1', 'rating': 4, 'caption': 'r사용자1|11-10|1', 'text': '직원분들이 친절해서 기분 좋게 식사했습니다.', 'seq': 1}}, {'r': {'reviewDate': neo4j.time.Date(2024, 1, 7), 'author': '사용자11', 'rating': 5, 'caption': 'r사용자11|01-07|1', 'text': '분위기가 좋아서 데이트 장소로 딱입니다.', 'seq': 1}}, {'r': {'reviewDate': neo4j.time.Date(2024, 1, 17), 'author': '사용자11', 'rating': 4, 'caption': 'r사용자11|01-17|1', 'text': '

'제공된 데이터에는 어떤 사용자가 티라미수를 구매했는지에 대한 정보가 없습니다. 따라서 티라미수를 구매한 사용자가 남긴 리뷰를 특정할 수 없습니다.'

In [46]:
chain.run("2024년 8월에 가장 많이 팔린 메뉴는 무엇인가요?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Purchase)-[:FOR_MENU_ITEM]->(m:MenuItem)
WHERE p.purchaseDate >= date('2024-08-01') AND p.purchaseDate < date('2024-09-01')
WITH m, sum(p.quantity) AS totalSold
RETURN m.name AS menuItem, totalSold
ORDER BY totalSold DESC
LIMIT 1;
Full Context:
[{'menuItem': '까르보나라 파스타', 'totalSold': 35}]

> Finished chain.


'까르보나라 파스타가 35개로 2024년 8월에 가장 많이 팔린 메뉴입니다.'

In [47]:
chain.run("글루텐 알레르기가 있는 고객이 먹을 수 없는 메뉴는 무엇인가요?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:MenuItem)-[:MAY_TRIGGER_ALLERGY]->(a:Allergy)
WHERE toLower(a.name) CONTAINS '글루텐'
RETURN DISTINCT m.name AS menuItem, m.category AS category, m.price AS price, m.currency AS currency;
Full Context:
[{'menuItem': '마르게리타 피자', 'category': '피자', 'price': 15000, 'currency': 'KRW'}, {'menuItem': '까르보나라 파스타', 'category': '파스타', 'price': 18000, 'currency': 'KRW'}]

> Finished chain.


'마르게리타 피자, 까르보나라 파스타는 글루텐 알레르기가 있는 고객이 먹을 수 없습니다.'

In [48]:
chain.run("2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 가장 많이 주문한 메뉴 카테고리는 무엇인가요?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (cr:CustomerReview)-[:WRITTEN_BY]->(c:Customer),
      (c)<-[:BY_CUSTOMER]-(p:Purchase)-[:FOR_MENU_ITEM]->(mi:MenuItem)-[:IN_MENU_CATEGORY]->(mc:MenuCategory)
WHERE cr.reviewDate >= date('2024-01-01') AND cr.reviewDate <= date('2024-06-30')
  AND cr.rating >= 4
  AND p.purchaseDate >= date('2024-01-01') AND p.purchaseDate <= date('2024-06-30')
RETURN mc.name AS category, SUM(p.quantity) AS totalQuantity
ORDER BY totalQuantity DESC
LIMIT 1
Full Context:
[{'category': '디저트', 'totalQuantity': 110}]

> Finished chain.


'디저트(총 110건)입니다.'

### 단계별로 프롬프트 주어서 답변 성능 올리기

In [ ]:
# 2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 가장 많이 주문한 메뉴 카테고리는 무엇인가요?

prompt = """당신은 복합 질의 쿼리를 잘 만드는 전문가 입니다. 단계별로 확인하고, 답변을 생성해주세요.
1. 2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 누구인지 확인.
2. 그 고객들이 주문한 메뉴를 집계
3. 메뉴, 집계 값 출력
"""
# 단계별로 프롬프트 주어서 답변 받기

In [50]:
chain.run(f"{prompt}")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (cr:CustomerReview)-[:WRITTEN_BY]->(c:Customer)
WHERE cr.reviewDate >= date('2024-01-01') AND cr.reviewDate <= date('2024-06-30') AND cr.rating >= 4
WITH DISTINCT c
MATCH (p:Purchase)-[:BY_CUSTOMER]->(c), (p)-[:FOR_MENU_ITEM]->(m:MenuItem)
RETURN m.name AS menu, sum(p.quantity) AS totalQuantity
ORDER BY totalQuantity DESC
Full Context:
[{'menu': '까르보나라 파스타', 'totalQuantity': 139}, {'menu': '마르게리타 피자', 'totalQuantity': 132}, {'menu': '티라미수', 'totalQuantity': 119}, {'menu': '아메리카노', 'totalQuantity': 108}]

> Finished chain.


'1) 2024년 상반기(1~6월)에 긍정적(4점 이상) 리뷰를 남긴 고객: 확인 불가\n\n2) 그 고객들이 주문한 메뉴 집계:\n- 까르보나라 파스타: 139\n- 마르게리타 피자: 132\n- 티라미수: 119\n- 아메리카노: 108\n\n3) 메뉴, 집계 값 출력:\n까르보나라 파스타 — 139  \n마르게리타 피자 — 132  \n티라미수 — 119  \n아메리카노 — 108'

---

### llm을 사용하여 cypher 쿼리 생성, 실행

In [51]:
question = "2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 누구인지 확인"

In [57]:
def generate_cypher_query_and_execute(question):
    llm = ChatOpenAI(model_name="gpt-5-mini")
    
    # 스키마 정보 가져오기
    schema = graph.schema
    
    # Cypher 쿼리 생성 프롬프트
    cypher_prompt = f"""
    Given the following Neo4j schema:
    {schema}
    Generate a Cypher query to answer this question: {question}
    Return only the Cypher query, no explanations.
    """
    
    # Cypher 쿼리 생성
    cypher_query = llm.predict(cypher_prompt).strip()
    
    # Cypher 쿼리 실행
    query_result = graph.query(cypher_query)
    return cypher_query, query_result

In [59]:
cypher_query, query_result = generate_cypher_query_and_execute(question)

In [60]:
cypher_query

"MATCH (cr:CustomerReview)-[:WRITTEN_BY]->(c:Customer)\nWHERE cr.rating >= 4\n  AND cr.reviewDate >= date('2024-01-01')\n  AND cr.reviewDate <= date('2024-06-30')\nRETURN DISTINCT c.name AS customer"

In [61]:
query_result

[{'customer': '사용자11'},
 {'customer': '사용자12'},
 {'customer': '사용자13'},
 {'customer': '사용자14'},
 {'customer': '사용자15'},
 {'customer': '사용자17'},
 {'customer': '사용자19'},
 {'customer': '사용자2'},
 {'customer': '사용자27'},
 {'customer': '사용자3'},
 {'customer': '사용자31'},
 {'customer': '사용자32'},
 {'customer': '사용자34'},
 {'customer': '사용자35'},
 {'customer': '사용자36'},
 {'customer': '사용자37'},
 {'customer': '사용자43'},
 {'customer': '사용자44'},
 {'customer': '사용자45'},
 {'customer': '사용자46'},
 {'customer': '사용자47'},
 {'customer': '사용자49'},
 {'customer': '사용자8'},
 {'customer': '사용자9'}]

---

In [ ]:
# schema 출력

print(graph.schema)

Node properties:
Restaurant {name: STRING, address: STRING, phone: STRING, hoursWeekday: STRING, hoursWeekend: STRING, seats: INTEGER, reservationAvailable: BOOLEAN, category: STRING}
Employee {name: STRING, experienceYears: INTEGER, role: STRING, workingDays: STRING}
Supplier {name: STRING, phone: STRING}
Customer {name: STRING}
MenuItem {name: STRING, category: STRING, price: INTEGER, currency: STRING}
MenuCategory {name: STRING}
Ingredient {name: STRING}
Allergy {name: STRING}
PaymentMethod {name: STRING}
Purchase {purchaseDate: DATE, quantity: INTEGER, totalPrice: INTEGER, pricePerUnit: INTEGER, customer: STRING, menuItem: STRING, seq: INTEGER, caption: STRING}
CustomerReview {reviewDate: DATE, rating: INTEGER, text: STRING, seq: INTEGER, author: STRING, caption: STRING}
ReservationChannel {name: STRING}
Relationship properties:

The relationships:
(:Restaurant)-[:ACCEPTS_PAYMENT_METHOD]->(:PaymentMethod)
(:Restaurant)-[:ACCEPTS_RESERVATION_CHANNEL]->(:ReservationChannel)
(:Restaur

---

In [64]:
question = "2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 누구인지 확인"

In [65]:
llm = ChatOpenAI(model_name="gpt-5-mini")

### 사용자 질문을 그래프 DB 조회에 필요한 단순한 탐색 경로/중간 데이터 보조 질문으로 분해

In [ ]:
# 사용자 질문을 그래프 DB 조회에 필요한 단순한 탐색 경로/중간 데이터 보조 질문으로 분해

import json
from typing import List, Dict, Tuple, Any
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph

def decompose_question_to_sub_queries(question: str, schema: str, llm: ChatOpenAI) -> List[str]:
    """
    LLM을 사용하여 복잡한 사용자 질문을 '그래프 DB 조회'에 필요한
    단순한 탐색 경로/중간 데이터 보조 질문으로 분해합니다.
    """
    decompose_prompt = f"""
    You are an expert Graph Query Decomposer. Your task is to analyze the user's complex question based ONLY on the provided Neo4j Schema.
    
    Decompose the question into a sequence of *data retrieval* steps (sub-questions). Each sub-question must target a specific piece of information from the graph. DO NOT ask about schema definition, delivery format, or user intent/policy.
    
    Neo4j Schema:
    ---
    {schema} 
    ---
    
    User Question: "{question}"
    
    Example Output: ["What is the name of the Head Chef?", "What menu items did the Head Chef create?", "Which ingredients are contained in those menu items?"]
    
    Return ONLY a JSON array of strings, where each string is a sub-question focused on data retrieval.
    """
    
    try:
        # LLM 호출 및 JSON 파싱
        # LLM의 응답은 JSON 문자열이라고 가정합니다.
        json_str = llm.invoke(decompose_prompt).content.strip()
        sub_questions = json.loads(json_str)
        
        # 리스트가 아니거나 형식이 맞지 않으면 빈 리스트 반환
        if not isinstance(sub_questions, list):
             return []
             
        return sub_questions
        
    except Exception as e:
        print(f"보조 질문 분해 중 오류 발생: {e}")
        return []

In [69]:
question

'2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 누구인지 확인'

In [71]:
# 예시 호출 (실제 실행 시 주석 해제 및 llm 객체 필요)
sub_queries = decompose_question_to_sub_queries(question, graph.schema, llm)
print(sub_queries)

['2024-01-01부터 2024-06-30 사이(reviewDate 범위)에 작성된 CustomerReview 노드 중 rating이 4 이상인 리뷰들은 어떤 것들인가?', '각 해당 CustomerReview 노드가 WRITTEN_BY 관계로 연결된 Customer 노드는 누구인가?', '각 해당 Customer 노드의 name 속성 값은 무엇인가?', '중복을 제거한(유일한) 2024년 상반기(1~6월)에 rating 4 이상인 리뷰를 남긴 고객 이름들의 목록은 무엇인가?']


### 보조 질문에 대한 쿼리 생성, 실행 및 결과

In [ ]:
# 보조 질문 실행 및 결과 context 생성 함수
def execute_sub_queries(question: str, sub_queries: List[str], graph: Neo4jGraph, llm: ChatOpenAI) -> Tuple[str, List[Dict[str, Any]]]:
    """
    보조 질문 목록을 순회하며 Cypher 쿼리를 생성 및 실행하고, 모든 결과를 Context로 수집합니다.
    """
    full_schema = graph.schema
    context_results = []
    
    # 각 보조 질문에 대한 쿼리를 생성하고 실행
    for i, sub_q in enumerate(sub_queries):
        # Cypher 쿼리 생성 프롬프트 (전체 스키마 사용)
        cypher_prompt = f"""
        Given the Neo4j schema and the following sub-question from a sequence of questions, generate the necessary Cypher query.
        
        Sub-Question {i+1}: "{sub_q}"
        
        Neo4j Schema:
        ---
        {full_schema}
        ---
        
        Return ONLY the valid Cypher query. Do not include any explanations.
        """
        
        try:
            # Cypher 쿼리 생성
            cypher_query = llm.invoke(cypher_prompt).content.strip()
            
            # Cypher 쿼리 실행
            query_result = graph.query(cypher_query)
            
            # 결과 Context 저장
            context_results.append({
                "sub_question": sub_q,
                "cypher_query": cypher_query,
                "result": query_result
            })
            
        except Exception as e:
            # 쿼리 생성 또는 실행 오류 시, 오류 메시지를 Context에 저장하여 LLM에게 전달
            context_results.append({
                "sub_question": sub_q,
                "error": f"Failed to execute query: {e}"
            })
            
    # 최종 답변을 위한 Context 문자열 생성 (프롬프트로 전달될 형태)
    context_string = f"Original User Question: {question}\n\nContextual Data Retrieved:\n"
    for item in context_results:
        if 'error' in item:
            context_string += f"- Q: {item['sub_question']} | Error: {item['error']}\n"
        else:
            # 결과가 너무 길면 잘라서 전달하거나 요약하는 로직을 추가할 수 있습니다.
            context_string += f"- Q: {item['sub_question']} | Result: {item['result']}\n"
            
    return context_string, context_results

In [73]:
# 예시 호출 (실제 실행 시 주석 해제 및 graph, llm 객체 필요)
final_context, detailed_results = execute_sub_queries(question, sub_queries, graph, llm)
print("--- LLM에게 전달할 최종 Context ---")
print(final_context)

--- LLM에게 전달할 최종 Context ---
Original User Question: 2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 누구인지 확인

Contextual Data Retrieved:
- Q: 2024-01-01부터 2024-06-30 사이(reviewDate 범위)에 작성된 CustomerReview 노드 중 rating이 4 이상인 리뷰들은 어떤 것들인가? | Result: [{'reviewDate': neo4j.time.Date(2024, 1, 5), 'rating': 4, 'text': '분위기가 좋아서 데이트 장소로 딱입니다.', 'author': '사용자17', 'caption': 'r사용자17|01-05|1', 'seq': 1}, {'reviewDate': neo4j.time.Date(2024, 1, 6), 'rating': 4, 'text': '티라미수가 정말 부드럽고 맛있어요.', 'author': '사용자27', 'caption': 'r사용자27|01-06|1', 'seq': 1}, {'reviewDate': neo4j.time.Date(2024, 1, 7), 'rating': 5, 'text': '분위기가 좋아서 데이트 장소로 딱입니다.', 'author': '사용자11', 'caption': 'r사용자11|01-07|1', 'seq': 1}, {'reviewDate': neo4j.time.Date(2024, 1, 7), 'rating': 4, 'text': '커피가 진하고 맛있습니다.', 'author': '사용자47', 'caption': 'r사용자47|01-07|1', 'seq': 1}, {'reviewDate': neo4j.time.Date(2024, 1, 11), 'rating': 4, 'text': '조용하고 아늑해서 혼자 책 읽기 좋아요.', 'author': '사용자45', 'caption': 'r사용자45|01-11|1', 'seq': 1}, {'reviewDate': neo4

In [75]:
final_context

"Original User Question: 2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 누구인지 확인\n\nContextual Data Retrieved:\n- Q: 2024-01-01부터 2024-06-30 사이(reviewDate 범위)에 작성된 CustomerReview 노드 중 rating이 4 이상인 리뷰들은 어떤 것들인가? | Result: [{'reviewDate': neo4j.time.Date(2024, 1, 5), 'rating': 4, 'text': '분위기가 좋아서 데이트 장소로 딱입니다.', 'author': '사용자17', 'caption': 'r사용자17|01-05|1', 'seq': 1}, {'reviewDate': neo4j.time.Date(2024, 1, 6), 'rating': 4, 'text': '티라미수가 정말 부드럽고 맛있어요.', 'author': '사용자27', 'caption': 'r사용자27|01-06|1', 'seq': 1}, {'reviewDate': neo4j.time.Date(2024, 1, 7), 'rating': 5, 'text': '분위기가 좋아서 데이트 장소로 딱입니다.', 'author': '사용자11', 'caption': 'r사용자11|01-07|1', 'seq': 1}, {'reviewDate': neo4j.time.Date(2024, 1, 7), 'rating': 4, 'text': '커피가 진하고 맛있습니다.', 'author': '사용자47', 'caption': 'r사용자47|01-07|1', 'seq': 1}, {'reviewDate': neo4j.time.Date(2024, 1, 11), 'rating': 4, 'text': '조용하고 아늑해서 혼자 책 읽기 좋아요.', 'author': '사용자45', 'caption': 'r사용자45|01-11|1', 'seq': 1}, {'reviewDate': neo4j.time.Date(2024, 1, 17),

In [78]:
detailed_results[0]['sub_question']
detailed_results[0]['cypher_query']

"MATCH (cr:CustomerReview)\nWHERE cr.reviewDate >= date('2024-01-01') AND cr.reviewDate <= date('2024-06-30') AND cr.rating >= 4\nRETURN cr.reviewDate AS reviewDate, cr.rating AS rating, cr.text AS text, cr.author AS author, cr.caption AS caption, cr.seq AS seq\nORDER BY cr.reviewDate;"

In [ ]:
detailed_results

[{'sub_question': '2024-01-01부터 2024-06-30 사이(reviewDate 범위)에 작성된 CustomerReview 노드 중 rating이 4 이상인 리뷰들은 어떤 것들인가?',
  'cypher_query': "MATCH (cr:CustomerReview)\nWHERE cr.reviewDate >= date('2024-01-01') AND cr.reviewDate <= date('2024-06-30') AND cr.rating >= 4\nRETURN cr.reviewDate AS reviewDate, cr.rating AS rating, cr.text AS text, cr.author AS author, cr.caption AS caption, cr.seq AS seq\nORDER BY cr.reviewDate;",
  'result': [{'reviewDate': neo4j.time.Date(2024, 1, 5),
    'rating': 4,
    'text': '분위기가 좋아서 데이트 장소로 딱입니다.',
    'author': '사용자17',
    'caption': 'r사용자17|01-05|1',
    'seq': 1},
   {'reviewDate': neo4j.time.Date(2024, 1, 6),
    'rating': 4,
    'text': '티라미수가 정말 부드럽고 맛있어요.',
    'author': '사용자27',
    'caption': 'r사용자27|01-06|1',
    'seq': 1},
   {'reviewDate': neo4j.time.Date(2024, 1, 7),
    'rating': 5,
    'text': '분위기가 좋아서 데이트 장소로 딱입니다.',
    'author': '사용자11',
    'caption': 'r사용자11|01-07|1',
    'seq': 1},
   {'reviewDate': neo4j.time.Date(2024, 1, 7),
    

### 보조 질문과 사이퍼 쿼리만 정리해서 출력

In [79]:
# 'sub_question'과 'cypher_query'만 정리해서 출력
summary_for_llm = [
    {
        "sub_question": item.get("sub_question"),
        "cypher_query": item.get("cypher_query")
    }
    for item in detailed_results
    if "sub_question" in item and "cypher_query" in item
]

In [80]:
summary_for_llm

[{'sub_question': '2024-01-01부터 2024-06-30 사이(reviewDate 범위)에 작성된 CustomerReview 노드 중 rating이 4 이상인 리뷰들은 어떤 것들인가?',
  'cypher_query': "MATCH (cr:CustomerReview)\nWHERE cr.reviewDate >= date('2024-01-01') AND cr.reviewDate <= date('2024-06-30') AND cr.rating >= 4\nRETURN cr.reviewDate AS reviewDate, cr.rating AS rating, cr.text AS text, cr.author AS author, cr.caption AS caption, cr.seq AS seq\nORDER BY cr.reviewDate;"},
 {'sub_question': '각 해당 CustomerReview 노드가 WRITTEN_BY 관계로 연결된 Customer 노드는 누구인가?',
  'cypher_query': 'MATCH (cr:CustomerReview)-[:WRITTEN_BY]->(c:Customer)\nRETURN cr, c\nORDER BY cr.seq'},
 {'sub_question': '각 해당 Customer 노드의 name 속성 값은 무엇인가?',
  'cypher_query': 'MATCH (c:Customer) RETURN c.name AS name;'},
 {'sub_question': '중복을 제거한(유일한) 2024년 상반기(1~6월)에 rating 4 이상인 리뷰를 남긴 고객 이름들의 목록은 무엇인가?',
  'cypher_query': 'MATCH (r:CustomerReview)-[:WRITTEN_BY]->(c:Customer)\nWHERE r.rating >= 4\n  AND r.reviewDate >= date("2024-01-01")\n  AND r.reviewDate <= date("2024-06-30")\

---

### 보조질문, 사이퍼쿼리를 참고하여 본 질문에 대한 사이퍼 쿼리 생성 및 실행

In [ ]:
# summary_for_llm(질문, 사이퍼쿼리) 를 참고하여 본 질문에 대한 cypher query 생성 및 실행

question = "2024년 상반기(1~6월)에 긍정적인 리뷰(4점 이상)를 남긴 고객들이 누구인지 확인"

def generate_cypher_query(question, summary_for_llm):
    llm = ChatOpenAI(model_name="gpt-5-mini")
    cypher_prompt = f"""
    아래는 참고할 수 있는 서브질문과 Cypher 쿼리 목록입니다:
    {summary_for_llm}
    
    위 정보를 참고하여, 다음 질문에 답할 수 있는 Cypher 쿼리를 생성하세요:
    {question}
    
    설명 없이 Cypher 쿼리만 반환하세요.
    """
    cypher_query = llm.predict(cypher_prompt).strip()
    
    query_result = graph.query(cypher_query)
    return cypher_query, query_result

cypher_query, query_result = generate_cypher_query(question, summary_for_llm)

In [85]:
cypher_query

'MATCH (r:CustomerReview)-[:WRITTEN_BY]->(c:Customer)\nWHERE r.rating >= 4\n  AND r.reviewDate >= date("2024-01-01")\n  AND r.reviewDate <= date("2024-06-30")\nRETURN DISTINCT c.name AS name\nORDER BY name;'

In [86]:
query_result

[{'name': '사용자11'},
 {'name': '사용자12'},
 {'name': '사용자13'},
 {'name': '사용자14'},
 {'name': '사용자15'},
 {'name': '사용자17'},
 {'name': '사용자19'},
 {'name': '사용자2'},
 {'name': '사용자27'},
 {'name': '사용자3'},
 {'name': '사용자31'},
 {'name': '사용자32'},
 {'name': '사용자34'},
 {'name': '사용자35'},
 {'name': '사용자36'},
 {'name': '사용자37'},
 {'name': '사용자43'},
 {'name': '사용자44'},
 {'name': '사용자45'},
 {'name': '사용자46'},
 {'name': '사용자47'},
 {'name': '사용자49'},
 {'name': '사용자8'},
 {'name': '사용자9'}]

---

### Cypher 생성을 위한 레스토랑 데이터베이스 특화 프롬프트를 활용하여 답변 성능 높이기

In [ ]:
import os
from langchain_core.prompts.prompt import PromptTemplate
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from typing import Dict, Any

from dotenv import load_dotenv
load_dotenv()

True

In [ ]:

# 설정 및 초기화 (실제 환경에 맞게 변경 필요)

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
USERNAME = os.environ.get("USERNAME")
PASSWORD = os.environ.get("PASSWORD")
URL = os.environ.get("URL")
llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)

NEO4J_URI = URL
graph = Neo4jGraph(url=NEO4J_URI, username=USERNAME, password=PASSWORD)

# LLM 초기화 (Cypher 생성 및 QA 답변 모두에 사용)
cypher_llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)
qa_llm = ChatOpenAI(model_name="gpt-5-mini", temperature=0)


In [96]:
graph.schema

'Node properties:\nRestaurant {name: STRING, address: STRING, phone: STRING, hoursWeekday: STRING, hoursWeekend: STRING, seats: INTEGER, reservationAvailable: BOOLEAN, category: STRING}\nEmployee {name: STRING, experienceYears: INTEGER, role: STRING, workingDays: STRING}\nSupplier {name: STRING, phone: STRING}\nCustomer {name: STRING}\nMenuItem {name: STRING, category: STRING, price: INTEGER, currency: STRING}\nMenuCategory {name: STRING}\nIngredient {name: STRING}\nAllergy {name: STRING}\nPaymentMethod {name: STRING}\nPurchase {purchaseDate: DATE, quantity: INTEGER, totalPrice: INTEGER, pricePerUnit: INTEGER, customer: STRING, menuItem: STRING, seq: INTEGER, caption: STRING}\nCustomerReview {reviewDate: DATE, rating: INTEGER, text: STRING, seq: INTEGER, author: STRING, caption: STRING}\nReservationChannel {name: STRING}\nRelationship properties:\n\nThe relationships:\n(:Restaurant)-[:ACCEPTS_PAYMENT_METHOD]->(:PaymentMethod)\n(:Restaurant)-[:ACCEPTS_RESERVATION_CHANNEL]->(:Reservation

In [ ]:

# 도메인 맞춤형 프롬프트 정의

# Cypher 생성을 위한 레스토랑 데이터베이스 특화 프롬프트
CYPHER_GENERATION_TEMPLATE = """Task: Generate Cypher statement to question a restaurant and customer review graph database (focused on Bella Roma).
Instructions:
- Use only the provided node labels, relationship types, and properties in the schema.
- Do not use any relationship types or properties not specified in the schema.
- Focus on extracting meaningful insights from restaurant, menu, purchase, and review data.

Schema:
{schema}

Note: 
- Provide only the Cypher statement.
- Do not include explanations or apologies.
- Generate precise, relevant Cypher queries.

Examples:
# 특정 직원의 근무일과 역할 조회
MATCH (e:Employee)
WHERE e.name = '김철수'
RETURN e.name, e.role, e.workingDays

# 특정 고객의 총 구매 횟수와 최고 구매액 조회
MATCH (c:Customer)<-[:BY_CUSTOMER]-(p:Purchase)
WHERE c.name = '사용자11'
RETURN c.name, COUNT(p) AS totalPurchases, MAX(p.totalPrice) AS maxPurchaseAmount

# 특정 메뉴의 재료와 유발 알러지 조회
MATCH (m:MenuItem {{name: '까르보나라 파스타'}})-[:CONTAINS_INGREDIENT]->(i:Ingredient)
OPTIONAL MATCH (m)-[:MAY_TRIGGER_ALLERGY]->(a:Allergy)
RETURN m.name, COLLECT(i.name) AS Ingredients, COLLECT(a.name) AS Allergies

# 가장 많이 팔린 메뉴 항목과 판매 수량 조회
MATCH (m:MenuItem)<-[:FOR_MENU_ITEM]-(p:Purchase)
RETURN m.name, SUM(p.quantity) AS TotalQuantity
ORDER BY TotalQuantity DESC
LIMIT 5

# 4점 이상 긍정적 리뷰를 가장 많이 남긴 고객 조회
MATCH (c:Customer)<-[:WRITTEN_BY]-(r:CustomerReview)
WHERE r.rating >= 4
RETURN c.name, COUNT(r) AS PositiveReviewCount
ORDER BY PositiveReviewCount DESC
LIMIT 10

The question is:
{question}"""

# 결과 처리를 위한 레스토랑 QA 프롬프트
QA_TEMPLATE = """
당신은 Bella Roma 레스토랑 데이터 분석 전문가로, 메뉴, 고객 행동 및 운영 정보에 대한 명확하고 간결한 정보를 한국어로 제공합니다.

[질문]
{question}

[검색 결과 (Cypher 쿼리 실행 결과)]
{context}

# 응답 가이드라인:
- 검색 결과에서 핵심 정보를 요약하세요
- 레스토랑 데이터에 대한 명확하고 객관적인 개요를 제공하세요
- 전문적이고 유익한 톤을 사용하세요
- 고객 행동, 인기 메뉴, 운영 효율성 측면에서 중요한 패턴이나 트렌드를 강조하세요
- 맥락이 불충분한 경우 더 많은 정보가 필요하다고 명확히 언급하세요
- 추측이나 개인적인 해석은 피하세요

# 응답 형식:
- 간략한 발견 요약으로 시작하세요
- 여러 항목이 발견된 경우 (예: 메뉴 목록, 고객 목록) 간결한 개요를 제공하세요
- 가독성을 위해 글머리 기호나 짧은 단락을 사용하세요
- 가격, 수량, 평점, 날짜와 같은 관련 수치 정보를 이해하기 쉬운 언어로 번역하세요

# 예시 응답 구조:
"분석 결과, [주요 발견 요약]

주요 특징/상세 정보:
- [첫 번째 중요 인사이트]
- [두 번째 중요 인사이트]

추가 정보: [필요한 경우 추가 설명]"
"""

# PromptTemplate 객체 생성
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE)

QA_PROMPT = PromptTemplate(
    input_variables=["question", "context"], 
    template=QA_TEMPLATE)

# --- 3. GraphCypherQAChain 생성 및 실행 함수 ---

def run_restaurant_qa_chain(
    question: str, 
    graph: Neo4jGraph, 
    cypher_llm: ChatOpenAI, 
    qa_llm: ChatOpenAI
) -> Dict[str, Any]:
    """
    레스토랑 데이터에 특화된 GraphCypherQAChain을 실행하고 결과를 반환합니다.
    """
    # graph.schema 값을 schema 인자로 전달
    cypher_chain = GraphCypherQAChain.from_llm(
        cypher_llm=cypher_llm,
        qa_llm=qa_llm,
        graph=graph, 
        allow_dangerous_requests=True,
        verbose=True, # Cypher 생성 과정을 확인하려면 True로 설정
        cypher_prompt=CYPHER_GENERATION_PROMPT,
        qa_prompt=QA_PROMPT,
        input_key="question",  
        output_key="result",
        # graph.schema를 cypher_prompt에 schema로 전달
        prompt_kwargs={"schema": graph.schema}
    )

    # Cypher 쿼리 실행
    print(f"\n[INFO] 질문 실행 중: {question}")
    answer = cypher_chain.invoke({"question": question})
    
    return answer


In [ ]:

# --- 4. 함수 실행 ---

question = "주방장 '김철수'가 일하는 레스토랑의 주중 영업시간을 알려주고, 어떤 결제 수단을 받는지 알려줘."
result = run_restaurant_qa_chain(question, graph, cypher_llm, qa_llm)
print("\n=== 최종 답변 ===")
print(result['result'])



[INFO] 질문 실행 중: 주방장 '김철수'가 일하는 레스토랑의 주중 영업시간을 알려주고, 어떤 결제 수단을 받는지 알려줘.


> Entering new GraphCypherQAChain chain...


Generated Cypher:
MATCH (e:Employee {name: '김철수'})-[:WORKS_AT_RESTAURANT]->(r:Restaurant)
OPTIONAL MATCH (r)-[:ACCEPTS_PAYMENT_METHOD]->(pm:PaymentMethod)
RETURN r.name, r.hoursWeekday, COLLECT(pm.name) AS acceptedPaymentMethods
Full Context:
[{'r.name': 'Bella Roma', 'r.hoursWeekday': '11:00-22:00', 'acceptedPaymentMethods': ['현금', '카드', '간편결제']}]

> Finished chain.

=== 최종 답변 ===
분석 결과, 주방장 '김철수'가 근무하는 레스토랑은 "Bella Roma"이며, 주중 영업시간은 11:00–22:00이고 허용 결제 수단은 현금, 카드, 간편결제로 등록되어 있습니다.

주요 특징/상세 정보:
- 레스토랑: Bella Roma (김철수 근무지)
- 주중 영업시간: 11:00–22:00 — 하루 기준 영업시간은 11시간입니다.
- 허용 결제수단: 현금, 카드, 간편결제
  - 데이터에선 "간편결제"로만 표기되어 있어 구체적인 서비스명(예: 특정 모바일 페이)은 제공되지 않았습니다.
- 데이터 범위 한계: 제공된 결과는 주중(hoursWeekday) 정보만 포함하고 있어 주말 영업시간, 휴무일, 카드 종류(신용/체크), 또는 간편결제의 상세 목록 등은 확인할 수 없습니다.

추가 정보:
- 주말/휴일 영업시간, 특정 결제 브랜드(예: Visa, MasterCard, Payco, 카카오페이 등) 확인을 원하시면 관련 필드(weekendHours, acceptedPaymentDetails 등)의 데이터 제공을 요청해 주세요.
- 고객 행동, 인기 메뉴, 운영 효율성(테이블 회전율, 주방 대기시간 등)에 대한 분석을 원하시면 해당 로그나 판매/예약 데이터가 필요합니다. 현재 결

In [ ]:
question = "‘가격’이라는 키워드가 포함된 리뷰들의 평균 평점은 얼마이며, 이 리뷰를 남긴 사용자들은 주로 어떤 메뉴를 주문했나요?"
result = run_restaurant_qa_chain(question, graph, cypher_llm, qa_llm)
print("\n=== 최종 답변 ===")
print(result['result'])

---

https://velog.io/@wltkqdl/%EC%A7%80%EC%8B%9D%EA%B7%B8%EB%9E%98%ED%94%84KG-%EA%B2%80%EC%83%89-Text2Cypher-%EA%B8%B0%EB%B2%95

In [104]:
# from langchain_openai import OpenAIEmbeddings
# from langchain_neo4j import Neo4jVector

# # 임베딩 모델 초기화
# embeddings = OpenAIEmbeddings(model="text-embedding-3-small") 

# # Neo4j 데이터베이스에 이미 생성된 벡터 인덱스에 연결하는 Neo4jVector 인스턴스 생성
# graph_db = Neo4jVector.from_existing_index(
#     embeddings,  # 사용할 임베딩 모델 지정
#     url=os.getenv("NEO4J_URI"),  # Neo4j 데이터베이스 연결 URI (환경 변수에서 가져옴)
#     username=os.getenv("NEO4J_USERNAME"),  # Neo4j 데이터베이스 사용자 이름
#     password=os.getenv("NEO4J_PASSWORD"),  # Neo4j 데이터베이스 비밀번호
#     index_name="movie_content_embeddings",  # 사용할 벡터 인덱스 이름 (이미 Neo4j에 생성되어 있어야 함)
#     text_node_property="overview",  # 텍스트 검색 시 반환할 노드의 속성 (영화 개요)
# )